# Cleaning scraped data

Import modules:

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

Data load and show first 5 rows:

In [3]:
raw_df = pd.read_csv(r"merged_golf.csv",
                     index_col =0,low_memory=False)
raw_df.head(n= 5)

,Unnamed: 0.1,ID,Price,Predict_Low,Predict_High,Location,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Liczba miejsc,Kolor,Możliwość finansowania,Faktura VAT,Kraj pochodzenia,Pierwsza rejestracja,Zarejestrowany w Polsce,Pierwszy właściciel,Bezwypadkowy,Serwisowany w ASO,Stan,Liczba drzwi,VAT marża,Numer rejestracyjny pojazdu,Wersja,Typ,Metalik,Leasing,Napęd,Okres gwarancji producenta,lub do (przebieg km),Perłowy,ABS,...,Elektrochromatyczne lusterko wsteczne,Gniazdo AUX,Kamera cofania,Podgrzewane lusterka boczne,System Start-Stop,CD,Elektrycznie ustawiane lusterka,Alarm,Asystent parkowania,Czujnik deszczu,Czujniki parkowania tylne,Gniazdo SD,Nawigacja GPS,Podgrzewane przednie siedzenia,Poduszki boczne tylne,Światła do jazdy dziennej,Alufelgi,Asystent pasa ruchu,Czujnik zmierzchu,Elektrochromatyczne lusterka boczne,Podgrzewana przednia szyba,Przyciemniane szyby,Tapicerka skórzana,Elektrycznie ustawiane fotele,Regulowane zawieszenie,Kurtyny powietrzne,Tempomat aktywny,Dach panoramiczny,Ogranicznik prędkości,Czujnik martwego pola,HUD (wyświetlacz przezierny),Podgrzewane tylne siedzenia,Odtwarzacz DVD,Ogrzewanie postojowe,Radio niefabryczne,Światła Xenonowe,Hak,Klimatyzacja czterostrefowa,Zmieniarka CD,Tuner TV
0,0,6.077535e+09,11900.0,11 250,13 100,NaN,Osoby prywatnej,Osobowe,Volkswagen,Golf,V (2003-2009),2006.0,210 000 km,1 900 cm3,Diesel,90 KM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6.070865e+09,14500.0,11 450,13 300,NaN,Osoby prywatnej,Osobowe,Volkswagen,Golf,V (2003-2009),2006.0,200 000 km,1 390 cm3,Benzyna+LPG,75 KM,NaN,5.0,Czarny,NaN,NaN,Niemcy,14/11/2006,Tak,NaN,Tak,NaN,Używane,3.0,NaN,OOL 51R5,V (2003-2009),NaN,Tak,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,6.076245e+09,81900.0,NaN,NaN,"Niciarniana 51/53 - 92-320 Łódź, Łódzkie (Polska)",Firmy,Osobowe,Volkswagen,Golf,VIII (2020-),2020.0,1 km,999 cm3,Benzyna,110 KM,Manualna,5.0,Biały,Tak,Tak,NaN,NaN,NaN,NaN,Tak,NaN,Nowe,5.0,NaN,NaN,NaN,Kompakt,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,6.072484e+09,68900.0,NaN,NaN,"Niciarniana 51/53 - 92-320 Łódź, Łódzkie (Polska)",Firmy,Osobowe,Volkswagen,Golf,VIII (2020-),2020.0,1 km,999 cm3,Benzyna,86 KM,NaN,5.0,Szary,Tak,Tak,NaN,NaN,NaN,NaN,Tak,NaN,Nowe,5.0,NaN,NaN,VII 1.0 TSI BMT Trendline,Kompakt,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,6.072485e+09,69500.0,NaN,NaN,"Niciarniana 51/53 - 92-320 Łódź, Łódzkie (Polska)",Firmy,Osobowe,Volkswagen,Golf,VIII (2020-),2020.0,1 km,999 cm3,Benzyna,86 KM,NaN,5.0,Biały,Tak,Tak,NaN,NaN,NaN,NaN,Tak,NaN,Nowe,5.0,NaN,NaN,VII 1.0 TSI BMT Trendline,Kompakt,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Processing of each column:

Create copy of dataframe:

In [4]:
clear_df = raw_df.copy()

Remove ID column:

In [5]:
clear_df = clear_df.drop(columns = 'ID')

Remove unamed column:

In [6]:
clear_df = clear_df.drop(columns = 'Unnamed: 0.1')
clear_df.head(1)

,Price,Predict_Low,Predict_High,Location,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Liczba miejsc,Kolor,Możliwość finansowania,Faktura VAT,Kraj pochodzenia,Pierwsza rejestracja,Zarejestrowany w Polsce,Pierwszy właściciel,Bezwypadkowy,Serwisowany w ASO,Stan,Liczba drzwi,VAT marża,Numer rejestracyjny pojazdu,Wersja,Typ,Metalik,Leasing,Napęd,Okres gwarancji producenta,lub do (przebieg km),Perłowy,ABS,Elektryczne szyby przednie,Poduszka powietrzna kierowcy,...,Elektrochromatyczne lusterko wsteczne,Gniazdo AUX,Kamera cofania,Podgrzewane lusterka boczne,System Start-Stop,CD,Elektrycznie ustawiane lusterka,Alarm,Asystent parkowania,Czujnik deszczu,Czujniki parkowania tylne,Gniazdo SD,Nawigacja GPS,Podgrzewane przednie siedzenia,Poduszki boczne tylne,Światła do jazdy dziennej,Alufelgi,Asystent pasa ruchu,Czujnik zmierzchu,Elektrochromatyczne lusterka boczne,Podgrzewana przednia szyba,Przyciemniane szyby,Tapicerka skórzana,Elektrycznie ustawiane fotele,Regulowane zawieszenie,Kurtyny powietrzne,Tempomat aktywny,Dach panoramiczny,Ogranicznik prędkości,Czujnik martwego pola,HUD (wyświetlacz przezierny),Podgrzewane tylne siedzenia,Odtwarzacz DVD,Ogrzewanie postojowe,Radio niefabryczne,Światła Xenonowe,Hak,Klimatyzacja czterostrefowa,Zmieniarka CD,Tuner TV
0,11900.0,11 250,13 100,NaN,Osoby prywatnej,Osobowe,Volkswagen,Golf,V (2003-2009),2006.0,210 000 km,1 900 cm3,Diesel,90 KM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop duplicates:

In [15]:
print(f'Numbers of rows before: {len(clear_df)}')
clear_df = clear_df.drop_duplicates()
print(f'Numbers of rows after: {len(clear_df)}')

Numbers of rows before: 9473
Numbers of rows after: 9105


Count empty rows in price column:

In [17]:
print('Empty rows with Price: ' + str(clear_df[clear_df['Price'].isnull()].sum(axis = 1).count()))

Empty rows with Price: 4589


Remove rows where all values are Nan:

In [18]:
print(f'Numbers of rows before: {len(clear_df)}')
clear_df = clear_df.dropna(how='all')
print(f'Numbers of rows after: {len(clear_df)}')

Numbers of rows before: 9105
Numbers of rows after: 9104


Empty cell in price column:

In [19]:
print('Empty rows with Price:' + str(clear_df[clear_df['Price'].isnull()].sum(axis = 1).count()))

Empty rows with Price:4588


Remove rows where price in unknown:

In [20]:
print(f'Numbers of rows before: {len(clear_df)}')
clear_df = clear_df.dropna(subset = ['Price'])
print(f'Numbers of rows after: {len(clear_df)}')

Numbers of rows before: 9104
Numbers of rows after: 4516


Convert columns: Predict_Low and Predict_High to int:

In [21]:
clear_df['Predict_Low'] = clear_df['Predict_Low'].fillna(0).apply(lambda x: int(str(x).replace(' ','')) if x is not None else x)
clear_df['Predict_High'] = clear_df['Predict_High'].fillna(0).apply(lambda x: int(str(x).replace(' ','')) if x is not None else x)

Create column with predicted average car price:

In [22]:
clear_df['Predict_average_price'] = (clear_df['Predict_Low'] + clear_df['Predict_High']) / 2
clear_df['Predict_average_price'] = clear_df['Predict_average_price'].astype('int')

Description of predicted average car price:

In [23]:
clear_df['Predict_average_price'].describe()

count      4516.000000
mean      30378.017050
std       32228.428816
min           0.000000
25%           0.000000
50%       20787.500000
75%       47850.000000
max      206850.000000
Name: Predict_average_price, dtype: float64

Remove columns Predict_Low and Predict_High:

In [24]:
clear_df = clear_df.drop(['Predict_Low','Predict_High'],axis = 1)

Convert column price into int:

In [25]:
clear_df['Price'] = clear_df['Price'].astype(int)

In [26]:
clear_df['Price'].describe()

count      4516.000000
mean      39238.979185
std       35371.009201
min         990.000000
25%       13999.000000
50%       25500.000000
75%       54900.000000
max      615000.000000
Name: Price, dtype: float64

Max value of Car is to high, show cars more expensive then 200 000:

In [27]:
clear_df[clear_df['Price']>200000]

,Price,Location,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Liczba miejsc,Kolor,Możliwość finansowania,Faktura VAT,Kraj pochodzenia,Pierwsza rejestracja,Zarejestrowany w Polsce,Pierwszy właściciel,Bezwypadkowy,Serwisowany w ASO,Stan,Liczba drzwi,VAT marża,Numer rejestracyjny pojazdu,Wersja,Typ,Metalik,Leasing,Napęd,Okres gwarancji producenta,lub do (przebieg km),Perłowy,ABS,Elektryczne szyby przednie,Poduszka powietrzna kierowcy,Wspomaganie kierownicy,ASR (kontrola trakcji),...,Gniazdo AUX,Kamera cofania,Podgrzewane lusterka boczne,System Start-Stop,CD,Elektrycznie ustawiane lusterka,Alarm,Asystent parkowania,Czujnik deszczu,Czujniki parkowania tylne,Gniazdo SD,Nawigacja GPS,Podgrzewane przednie siedzenia,Poduszki boczne tylne,Światła do jazdy dziennej,Alufelgi,Asystent pasa ruchu,Czujnik zmierzchu,Elektrochromatyczne lusterka boczne,Podgrzewana przednia szyba,Przyciemniane szyby,Tapicerka skórzana,Elektrycznie ustawiane fotele,Regulowane zawieszenie,Kurtyny powietrzne,Tempomat aktywny,Dach panoramiczny,Ogranicznik prędkości,Czujnik martwego pola,HUD (wyświetlacz przezierny),Podgrzewane tylne siedzenia,Odtwarzacz DVD,Ogrzewanie postojowe,Radio niefabryczne,Światła Xenonowe,Hak,Klimatyzacja czterostrefowa,Zmieniarka CD,Tuner TV,Predict_average_price
1861,219900,"Uczniowska 36 - 80-001 Gdańsk, Letnica (Polska)",Firmy,Osobowe,Volkswagen,Golf,VII (2012-2020),2019.0,1 km,1 989 cm3,Benzyna,300 KM,Automatyczna,5.0,Szary,Tak,Tak,Polska,NaN,NaN,NaN,Tak,NaN,Nowe,5.0,NaN,NaN,NaN,Kompakt,Tak,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,...,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,206850
2734,615000,NaN,Firmy,Osobowe,Volkswagen,Golf,NaN,2016.0,221 000 km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VII (2012-),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2948,400000,NaN,Firmy,Osobowe,Volkswagen,Golf,IV (1997-2006),2018.0,1 000 000 km,999 cm3,Benzyna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VII 1.0 TSI BMT City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Remove 2 the most expensive cars on list:

In [28]:
clear_df = clear_df[clear_df.Price < 220000]

Create temporary column with location:

In [30]:
clear_df['Location_TEST'] = clear_df['Location']

Function to process address:

In [31]:
def Location (x):
    '''
    Function removes the street name and house number from the column.
    Keep only the City, region, and Country name.
    If the location is unknown assign Poland.
    '''
    if x is np.NAN:
        return 'Poland'
    else:
        temp_list = x.split(' ')
        numbers_of_elements = len(temp_list)
        if numbers_of_elements > 3:
            return " ".join(temp_list[numbers_of_elements-3:])
        else:
            return x

Run function in Location_TEST column:

In [32]:
clear_df['Location_TEST'] = clear_df['Location_TEST'].apply(Location)

Convert Location address to Latitude and Longitude:

In [35]:
import geopandas
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="data_cleaning")
locator = Nominatim(user_agent="myGeocoder")

In [36]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=0)
# 2- - create location column
clear_df['Cordinates'] = clear_df['Location_TEST'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
clear_df['point'] = clear_df['Cordinates'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
clear_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(clear_df['point'].tolist(), index=clear_df.index)

Remove from database empty cells with locations:

In [37]:
clear_df = clear_df[clear_df['longitude'].notnull()]

Remove unusued columns of locations:

In [38]:
clear_df = clear_df.drop(columns = ['Location','Location_TEST','Cordinates','altitude','point'])

Save/load dataframe to csv after location processing:

In [42]:
clear_df.to_csv('Locaton_Golf.csv')

In [138]:
clear_df = pd.read_csv('Locaton_Golf.csv')

Function to change seller name from Polish to English:

In [139]:
def seller(x):
    if x == 'Osoby prywatnej':
        return 1
    else:
        return 0

Apply seller function to new column and remove old one:

In [140]:
clear_df['private-seller'] = clear_df['Oferta od'].apply(seller)
clear_df = clear_df.drop(columns = ['Oferta od'])

Remove column kategoria because keep everywhere the same value:

In [141]:
clear_df = clear_df.drop(columns = ['Kategoria'])

Remove null values in car generation from dataframe:

In [142]:
clear_df = clear_df[clear_df['Generacja'].notnull()]

Convert year of production to int:

In [143]:
clear_df['Rok produkcji'] = clear_df['Rok produkcji'].astype(int)

Convert distanse from str to int:

In [144]:
clear_df['distance'] = clear_df['Przebieg'].apply(lambda x: (str(x)).replace(' ','').replace('km',''))

In [145]:
clear_df['distance'] = clear_df['distance'].astype(float)
clear_df['distance'] = clear_df['distance'].fillna(0)
clear_df['distance'] = clear_df['distance'].astype(int)

In [146]:
print(f'Numbers of rows before: {len(clear_df)}')
clear_df = clear_df.drop(columns = ['Przebieg'])
print(f'Numbers of rows after: {len(clear_df)}')

Numbers of rows before: 4309
Numbers of rows after: 4309


Transform colum Pojemność skokowa into engine-capacity type of int:

In [147]:
clear_df['engine-capacity'] = clear_df['Pojemność skokowa'].apply(lambda x: (str(x)).replace(' ','').replace('cm3',''))
clear_df['engine-capacity'] = clear_df['engine-capacity'].astype(float)
clear_df = clear_df[clear_df['engine-capacity'].notnull()]
clear_df['engine-capacity'] = clear_df['engine-capacity'].astype(int)
clear_df = clear_df.drop(columns = ['Pojemność skokowa'])

Transform column Rodzaj paliwa into 0,1 values in separated columns:

In [148]:
clear_df = clear_df[(clear_df['Rodzaj paliwa'] == 'Benzyna') | (clear_df['Rodzaj paliwa'] == 'Diesel')]
clear_df['Petrol'] = clear_df['Rodzaj paliwa'].apply(lambda x : 1 if x == 'Benzyna' else 0)
clear_df['Diesel'] = clear_df['Rodzaj paliwa'].apply(lambda x : 1 if x == 'Diesel' else 0)
clear_df = clear_df.drop(columns = 'Rodzaj paliwa')

Transform column Moc into engine-output with int values. Remove empty rows and column Moc:

In [149]:
clear_df = clear_df[clear_df['Moc'].notnull()]
clear_df['engine-output'] = clear_df['Moc'].apply(lambda x : x[:-3] if x.endswith(' KM') else 0)
clear_df['engine-output'] = clear_df['engine-output'].astype(int)
clear_df = clear_df.drop(columns = 'Moc')

Transform column Skrzynia biegów into Automatic transmission with 0/1 values:

In [150]:
clear_df['Skrzynia biegów'].value_counts()

Manualna        1671
Automatyczna     326
Name: Skrzynia biegów, dtype: int64

In [151]:
clear_df['Skrzynia biegów'] = clear_df['Skrzynia biegów'].fillna("Manualna")
clear_df['automatic-transmission'] = clear_df['Skrzynia biegów'].apply(lambda x : 1 if x == 'Automatyczna' else 0)
clear_df['automatic-transmission'] = clear_df['automatic-transmission'].astype(int)
clear_df = clear_df.drop(columns = 'Skrzynia biegów')

From column Liczba miejsc create int column with numbers of seats fill empty with value =5 and remove column Liczba miejsc:

In [152]:
clear_df['Liczba miejsc'] = clear_df['Liczba miejsc'].fillna(5)
clear_df['number-of-seats'] = clear_df['Liczba miejsc']
clear_df['number-of-seats'] = clear_df['number-of-seats'].astype(int)
clear_df = clear_df.drop(columns = 'Liczba miejsc')

1. Create column color.
2. Remove empty rows with Colour.
3. Translate values in column Color.

In [153]:
eng_colors = '''Black 1003
Silver 743
Gray 596
White 466
Blue 442
Other_color 194
Red 125
Green 51
Gold 39
Beige 38
Yellow 27
Burgundy 27
Brown 26
Purple 15'''

eng_colors_list = []
eng_colors =  eng_colors.split('\n')
for item in eng_colors:
    eng_colors_list.append(item.split(" ")[0])


pl_colors = '''Czarny        1003
Srebrny        743
Szary          596
Biały          466
Niebieski      442
Inny kolor     194
Czerwony       125
Zielony         51
Złoty           39
Beżowy          38
Żółty           27
Bordowy         27
Brązowy         26
Fioletowy       15'''

pl_colors_list = []
pl_colors =  pl_colors.split('\n')
for item in pl_colors:
    pl_colors_list.append(item.split(" ")[0]) if item.split(" ")[0] != 'Inny' else pl_colors_list.append("Inny kolor")

    
clear_df['color'] = clear_df['Kolor'].replace(dict(zip(pl_colors_list, eng_colors_list)) , inplace=True)
clear_df['color'] = clear_df['Kolor']
clear_df = clear_df[clear_df['color'].notnull()]
clear_df = clear_df.drop(columns = ['Kolor'])

1. Create column finance option,
2. Append with 1 if finnancing is posible 0 if not
3. Remove column Możliwość finansowania

In [154]:
clear_df['funding-available'] = clear_df['Możliwość finansowania']
clear_df['funding-available'] = clear_df['funding-available'].apply(lambda x: 1 if x == 'Tak' else 0)
clear_df = clear_df.drop(columns = ['Możliwość finansowania'])

1. Create column Invoice-for-car,
2. Append with 1 if Invoice is posible 0 if not
3. Remove column Faktura VAT

In [155]:
clear_df['Invoice-for-car'] = clear_df['Faktura VAT']
clear_df['Invoice-for-car'] = clear_df['Invoice-for-car'].apply(lambda x: 1 if x == 'Tak' else 0)
clear_df = clear_df.drop(columns = ['Faktura VAT'])

1. Create column Bought as new in Poland,
2. Append with 1 if Yes is posible 0 if not
3. Remove column Kraj pochodzenia

In [156]:
clear_df['Bought-as-new-in-Poland'] = clear_df['Kraj pochodzenia']
clear_df['Bought-as-new-in-Poland'] = clear_df['Bought-as-new-in-Poland'].apply(lambda x: 1 if x == 'Polska' else 0)
clear_df = clear_df.drop(columns = ['Kraj pochodzenia'])

Remove column Pierwsza rejestracja because lack of values and strongly correlated with year of production:

In [157]:
clear_df = clear_df.drop(columns = ['Pierwsza rejestracja'])

1. Create column Registered in Poland
2. Append with 1 if Yes is posible 0 if not
3. Remove column Zarejestrowany w Polsce

In [158]:
clear_df['Registered_in_Poland'] = clear_df['Zarejestrowany w Polsce']
clear_df['Registered_in_Poland'] = clear_df['Registered_in_Poland'].apply(lambda x: 1 if x == 'Tak' else 0)
clear_df = clear_df.drop(columns = ['Zarejestrowany w Polsce'])

1. Create column Never crashed
2. Append with 1 if Yes is posible 0 if not
3. Remove column Bezwypadkowy

In [159]:
clear_df['Never crashed'] = clear_df['Bezwypadkowy']
clear_df['Never crashed'] = clear_df['Never crashed'].apply(lambda x: 1 if x == 'Tak' else 0)
clear_df = clear_df.drop(columns = ['Bezwypadkowy'])

1. Create column From first owner
2. Append with 1 if Yes is posible 0 if not
3. Remove column Pierwszy właściciel

In [160]:
clear_df['From-first-owner'] = clear_df['Pierwszy właściciel']
clear_df['From-first-owner'] = clear_df['From-first-owner'].apply(lambda x: 1 if x == 'Tak' else 0)
clear_df = clear_df.drop(columns = ['Pierwszy właściciel'])

1. Create column Serviced in authorized service center
2. Append with 1 if Yes is posible 0 if not
3. Remove column Serwisowany w ASO

In [161]:
clear_df['Serviced in authorized service center'] = clear_df['Serwisowany w ASO']
clear_df['Serviced in authorized service center'] = clear_df['Serviced in authorized service center'].apply(lambda x: 1 if x == 'Tak' else 0)
clear_df = clear_df.drop(columns = ['Serwisowany w ASO'])

Remove brand new cars from data base:

In [162]:
clear_df = clear_df[clear_df.Stan != 'Nowe']

Remove column Stan:

In [163]:
clear_df = clear_df.drop(columns = ['Stan'])

Remove column Liczba drzwi because most car in list is 5 door:

In [164]:
clear_df = clear_df.drop(columns = ['Liczba drzwi'])

Rename converted columns:

In [165]:
clear_df = clear_df.rename(columns={"Marka pojazdu": "Brand", "Model pojazdu": "Model"})
clear_df.head(1)

,Unnamed: 0,Price,Brand,Model,Generacja,Rok produkcji,VAT marża,Numer rejestracyjny pojazdu,Wersja,Typ,Metalik,Leasing,Napęd,Okres gwarancji producenta,lub do (przebieg km),Perłowy,ABS,Elektryczne szyby przednie,Poduszka powietrzna kierowcy,Wspomaganie kierownicy,ASR (kontrola trakcji),Elektryczne szyby tylne,Gniazdo USB,Klimatyzacja dwustrefowa,MP3,Poduszka powietrzna chroniąca kolana,Relingi dachowe,Światła LED,Tempomat,Klimatyzacja manualna,Tapicerka welurowa,Centralny zamek,Komputer pokładowy,Szyberdach,Wielofunkcyjna kierownica,Isofix,ESP (stabilizacja toru jazdy),Poduszka powietrzna pasażera,Łopatki zmiany biegów,Radio fabryczne,...,Elektrochromatyczne lusterka boczne,Podgrzewana przednia szyba,Przyciemniane szyby,Tapicerka skórzana,Elektrycznie ustawiane fotele,Regulowane zawieszenie,Kurtyny powietrzne,Tempomat aktywny,Dach panoramiczny,Ogranicznik prędkości,Czujnik martwego pola,HUD (wyświetlacz przezierny),Podgrzewane tylne siedzenia,Odtwarzacz DVD,Ogrzewanie postojowe,Radio niefabryczne,Światła Xenonowe,Hak,Klimatyzacja czterostrefowa,Zmieniarka CD,Tuner TV,Predict_average_price,latitude,longitude,private-seller,distance,engine-capacity,Petrol,Diesel,engine-output,automatic-transmission,number-of-seats,color,funding-available,Invoice-for-car,Bought-as-new-in-Poland,Registered_in_Poland,Never crashed,From-first-owner,Serviced in authorized service center
6,6,15900,Volkswagen,Golf,V (2003-2009),2008,NaN,NaN,NaN,Kompakt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,52.215933,19.134422,1,200000,1390,1,0,140,0,5,Black,1,0,1,1,1,0,0


Transform generation column:

In [166]:
clear_df['Generacja'].value_counts(dropna = False)

VII (2012-2020)    1207
VI (2008-2013)      846
V (2003-2009)       791
IV (1997-2006)      401
III (1991-1998)      33
II (1983-1992)       30
VIII (2020-)         17
I (1974-1983)        10
Name: Generacja, dtype: int64

In [167]:
clear_df['Generacja']= clear_df['Generacja'].apply(lambda x: x.split(' ')[0])
clear_df['Generacja']

6         V
12      VII
13      VII
17      VII
18      VII
       ... 
4507    VII
4508      V
4509     VI
4510    VII
4511     IV
Name: Generacja, Length: 3335, dtype: object

In [168]:
clear_df = clear_df.rename(columns={"Generacja": "Generation"})
clear_df.head(1)

,Unnamed: 0,Price,Brand,Model,Generation,Rok produkcji,VAT marża,Numer rejestracyjny pojazdu,Wersja,Typ,Metalik,Leasing,Napęd,Okres gwarancji producenta,lub do (przebieg km),Perłowy,ABS,Elektryczne szyby przednie,Poduszka powietrzna kierowcy,Wspomaganie kierownicy,ASR (kontrola trakcji),Elektryczne szyby tylne,Gniazdo USB,Klimatyzacja dwustrefowa,MP3,Poduszka powietrzna chroniąca kolana,Relingi dachowe,Światła LED,Tempomat,Klimatyzacja manualna,Tapicerka welurowa,Centralny zamek,Komputer pokładowy,Szyberdach,Wielofunkcyjna kierownica,Isofix,ESP (stabilizacja toru jazdy),Poduszka powietrzna pasażera,Łopatki zmiany biegów,Radio fabryczne,...,Elektrochromatyczne lusterka boczne,Podgrzewana przednia szyba,Przyciemniane szyby,Tapicerka skórzana,Elektrycznie ustawiane fotele,Regulowane zawieszenie,Kurtyny powietrzne,Tempomat aktywny,Dach panoramiczny,Ogranicznik prędkości,Czujnik martwego pola,HUD (wyświetlacz przezierny),Podgrzewane tylne siedzenia,Odtwarzacz DVD,Ogrzewanie postojowe,Radio niefabryczne,Światła Xenonowe,Hak,Klimatyzacja czterostrefowa,Zmieniarka CD,Tuner TV,Predict_average_price,latitude,longitude,private-seller,distance,engine-capacity,Petrol,Diesel,engine-output,automatic-transmission,number-of-seats,color,funding-available,Invoice-for-car,Bought-as-new-in-Poland,Registered_in_Poland,Never crashed,From-first-owner,Serviced in authorized service center
6,6,15900,Volkswagen,Golf,V,2008,NaN,NaN,NaN,Kompakt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,52.215933,19.134422,1,200000,1390,1,0,140,0,5,Black,1,0,1,1,1,0,0


Convert column Vat marża to VAT margin:

In [169]:
clear_df = clear_df.rename(columns={"VAT marża": "VAT margin"})
clear_df.head(1)

,Unnamed: 0,Price,Brand,Model,Generation,Rok produkcji,VAT margin,Numer rejestracyjny pojazdu,Wersja,Typ,Metalik,Leasing,Napęd,Okres gwarancji producenta,lub do (przebieg km),Perłowy,ABS,Elektryczne szyby przednie,Poduszka powietrzna kierowcy,Wspomaganie kierownicy,ASR (kontrola trakcji),Elektryczne szyby tylne,Gniazdo USB,Klimatyzacja dwustrefowa,MP3,Poduszka powietrzna chroniąca kolana,Relingi dachowe,Światła LED,Tempomat,Klimatyzacja manualna,Tapicerka welurowa,Centralny zamek,Komputer pokładowy,Szyberdach,Wielofunkcyjna kierownica,Isofix,ESP (stabilizacja toru jazdy),Poduszka powietrzna pasażera,Łopatki zmiany biegów,Radio fabryczne,...,Elektrochromatyczne lusterka boczne,Podgrzewana przednia szyba,Przyciemniane szyby,Tapicerka skórzana,Elektrycznie ustawiane fotele,Regulowane zawieszenie,Kurtyny powietrzne,Tempomat aktywny,Dach panoramiczny,Ogranicznik prędkości,Czujnik martwego pola,HUD (wyświetlacz przezierny),Podgrzewane tylne siedzenia,Odtwarzacz DVD,Ogrzewanie postojowe,Radio niefabryczne,Światła Xenonowe,Hak,Klimatyzacja czterostrefowa,Zmieniarka CD,Tuner TV,Predict_average_price,latitude,longitude,private-seller,distance,engine-capacity,Petrol,Diesel,engine-output,automatic-transmission,number-of-seats,color,funding-available,Invoice-for-car,Bought-as-new-in-Poland,Registered_in_Poland,Never crashed,From-first-owner,Serviced in authorized service center
6,6,15900,Volkswagen,Golf,V,2008,NaN,NaN,NaN,Kompakt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,52.215933,19.134422,1,200000,1390,1,0,140,0,5,Black,1,0,1,1,1,0,0


In [170]:
clear_df['VAT margin'].value_counts(dropna = False)

NaN    2147
Tak    1188
Name: VAT margin, dtype: int64

In [171]:
clear_df['VAT margin'] = clear_df['VAT margin'].apply(lambda x: 1 if x == "Tak" else 0)
clear_df['VAT margin'].value_counts(dropna = False)

0    2147
1    1188
Name: VAT margin, dtype: int64

Transform column production year:

In [172]:
clear_df = clear_df.rename(columns={"Rok produkcji": "Year-of-production"})
clear_df['Year-of-production'].value_counts(dropna = False)

2009    303
2016    234
2010    234
2017    211
2008    201
2015    196
2011    192
2007    176
2018    162
2013    149
2014    148
2004    137
2005    132
2019    127
2006    123
2012    123
2003     80
2000     69
1999     68
2001     68
2002     65
1998     41
2020     25
1991     13
1997      9
1996      9
1990      9
1995      7
1989      4
1994      4
1986      4
1993      2
1992      2
1988      2
1984      2
1979      1
1985      1
1987      1
1980      1
Name: Year-of-production, dtype: int64

Drop column Numer rejestracyjny pojazdu:


In [173]:
clear_df = clear_df.drop(columns = ['Numer rejestracyjny pojazdu'])

Drop column Wersja:

In [174]:
clear_df = clear_df.drop(columns = ['Wersja'])

Delete column Typ
because > 50% Null:


In [175]:
clear_df = clear_df.drop(columns = ['Typ'])

Transform column Metalic what mean metalic type of paiting to 1 if Yes to 0 if No:

In [176]:
clear_df = clear_df.rename(columns={"Metalik": "Metalic"})
clear_df['Metalic'].value_counts(dropna = False)

Tak    2221
NaN    1114
Name: Metalic, dtype: int64

In [177]:
clear_df['Metalic'] = clear_df['Metalic'].apply(lambda x: 1 if x=='Tak' else 0)
clear_df['Metalic'].value_counts(dropna = False)

1    2221
0    1114
Name: Metalic, dtype: int64

Transform column Leasing:
1. Change values to 1 if leasing is possible
2. Change values to 0 if leasing is not possible or unknow

In [178]:
clear_df['Leasing'].value_counts(dropna = False)

NaN    3024
Tak     311
Name: Leasing, dtype: int64

In [179]:
clear_df['Leasing'] = clear_df['Leasing'].apply(lambda x: 1 if x=='Tak' else 0)
clear_df['Leasing'].value_counts(dropna = False)

0    3024
1     311
Name: Leasing, dtype: int64

Transform column Napęd:
1. Change column name to All_wheel_drive

In [180]:
clear_df = clear_df.rename(columns={"Napęd": "All_wheel_drive"})
clear_df['All_wheel_drive'] = clear_df['All_wheel_drive'].apply(lambda x: 1 if x=='4x4 (dołączany automatycznie)' or  x == '4x4 (stały)' else 0)
clear_df['All_wheel_drive'].value_counts(dropna = False)

0    3306
1      29
Name: All_wheel_drive, dtype: int64

Drop column Okres gwarancji producenta:


In [190]:
clear_df = clear_df.drop(columns=["Okres gwarancji producenta"])

KeyError: ignored

Transform column lub do (przebieg km):
1. If value is different then Nan that mean the car have a warranty
2. If cars have warranty assign 1 else 0

In [220]:
clear_df['lub do (przebieg km)'] = clear_df['lub do (przebieg km)'].fillna(0)
clear_df['lub do (przebieg km)'] = clear_df['lub do (przebieg km)'].apply(lambda x: 0 if x == 0 else 1)
clear_df = clear_df.rename(columns={"lub do (przebieg km)": "On_warranty"})
clear_df["On_warranty"].value_counts()

0    3276
1      59
Name: On_warranty, dtype: int64

Create list of columns from df where are car features and 1.0 means car have feature and NaN not:


In [228]:
col_list = clear_df.columns.to_list()[11:-19]
col_list

['Perłowy',
 'ABS',
 'Elektryczne szyby przednie',
 'Poduszka powietrzna kierowcy',
 'Wspomaganie kierownicy',
 'ASR (kontrola trakcji)',
 'Elektryczne szyby tylne',
 'Gniazdo USB',
 'Klimatyzacja dwustrefowa',
 'MP3',
 'Poduszka powietrzna chroniąca kolana',
 'Relingi dachowe',
 'Światła LED',
 'Tempomat',
 'Klimatyzacja manualna',
 'Tapicerka welurowa',
 'Centralny zamek',
 'Komputer pokładowy',
 'Szyberdach',
 'Wielofunkcyjna kierownica',
 'Isofix',
 'ESP (stabilizacja toru jazdy)',
 'Poduszka powietrzna pasażera',
 'Łopatki zmiany biegów',
 'Radio fabryczne',
 'Klimatyzacja automatyczna',
 'Poduszki boczne przednie',
 'Światła przeciwmgielne',
 'Immobilizer',
 'Bluetooth',
 'Czujniki parkowania przednie',
 'Elektrochromatyczne lusterko wsteczne',
 'Gniazdo AUX',
 'Kamera cofania',
 'Podgrzewane lusterka boczne',
 'System Start-Stop',
 'CD',
 'Elektrycznie ustawiane lusterka',
 'Alarm',
 'Asystent parkowania',
 'Czujnik deszczu',
 'Czujniki parkowania tylne',
 'Gniazdo SD',
 'Nawiga

In [246]:
for col_name in col_list:
    clear_df[col_name] = clear_df[col_name].replace('Tak',1).fillna(0)

In [248]:
for col_name in col_list:
    clear_df[col_name] = clear_df[col_name].astype(int)

Translate columns to English:

In [250]:
translated = ['Pearl-paint',
 'ABS',
 'Electric front windows',
 'Driver airbag',
 'Power steering',
 'ASR',
 'Electric rear windows',
 'USB',
 'Dual-zone-air-conditioning',
 'MP3',
 'Knee-protection-airbag',
 'Roof-rails',
 'LED-lights',
 'Cruise-control',
 'Manual-air-conditioning',
 'Velor-upholstery',
 'Central-lock',
 'On-board-computer',
 'Sunroof',
 'Multifunction-steering-wheel',
 'Isofix',
 'ESP',
 'Passenger-airbag',
 'Gear-shift-paddles',
 'Factory-radio',
 'Automatic-air-conditioning',
 'Front-side-airbags',
 'Fog-lights',
 'Immobilizer',
 'Bluetooth',
 'Front-parking-sensors',
 'Electrochromic-rear-view-mirror',
 'AUX',
 'Rear-view-camera',
 'Heated-door-mirrors',
 'Start-Stop-system',
 'CD',
 'Electrically-adjustable-mirrors',
 'Alarm',
 'Parking-assistant',
 'Rain-sensor',
 'Rear-parking-sensors',
 'SD-slot',
 'GPS-navigation',
 'Heated-front-seats',
 'Rear-side-airbags',
 'Daytime-running-lights',
 'Alloy-wheels',
 'Lane-Assistant',
 'Dusk-sensor',
 'Electrochromic-side-mirrors',
 'Heated-windscreen',
 'Tinted-windows',
 'Leather-upholstery',
 'Electrically-adjustable-seats',
 'Adjustable-suspension',
 'Air-curtains',
 'Cruise-control-active',
 'Panoramic-roof',
 'Speed-limiter',
 'Blind-spot-sensor',
 'HUD',
 'Heated-rear-seats',
 'DVD-player',
 'Parking-heating',
 'Non-factory-radio',
 'Xenon-lights',
 'Hook',
 'Four-zone-air-conditioning',
 'CD-changer',
 'TV-tuner']

In [264]:
clear_df = clear_df.rename(columns=dict(zip(col_list,translated)) )

In [ ]:
clear_df = clear_df.drop(columns=['Unnamed: 0'])

In [284]:
clear_df.reset_index(inplace=True,drop= True)

Return information about columns

In [291]:
clear_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3335 entries, 0 to 3334
Data columns (total 100 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Price                                  3335 non-null   int64  
 1   Brand                                  3335 non-null   object 
 2   Model                                  3335 non-null   object 
 3   Generation                             3335 non-null   object 
 4   Year-of-production                     3335 non-null   int64  
 5   VAT margin                             3335 non-null   int64  
 6   Metalic                                3335 non-null   int64  
 7   Leasing                                3335 non-null   int64  
 8   All_wheel_drive                        3335 non-null   int64  
 9   On_warranty                            3335 non-null   int64  
 10  Pearl-paint                            3335 non-null   int64  
 11  ABS

Drop objects type columns:

In [292]:
ml_df = clear_df.copy()
ml_df = ml_df.drop(columns= ['color','Brand','Generation','Model'])

Save file to CSV

In [296]:
ml_df.to_csv('Golf_data_to_ml.csv')

Download the data:

In [297]:
from google.colab import files
files.download("Golf_data_to_ml.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save cleared golf data to csv and download it:

In [299]:
clear_df.to_csv('Golf_data_clear.csv')
from google.colab import files
files.download("Golf_data_clear.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>